In [1]:
from IPython.core.display import display, HTML
display(HTML("""<style> .container {width:96% !important;}</style>"""))

from IPython.display import IFrame

In [2]:
import pandas as pd
import numpy as np
# from plotly.offline import init_notebook_mode, iplot
# import cufflinks as cf
# init_notebook_mode()
# cf.go_offline()
from __future__ import division

In [3]:
import xgboost as xgb
import sys
sys.path.insert(0,'../')
from utils.paths import *

# Explain the grades

![](asset/Explanation.png)

In [6]:
!aws s3 ls s3://eh-home/ehda-calvin/SBA_study/

                           PRE pdf/
2018-10-29 08:48:12      16185 7a_504_FOIA Data Dictionary.xlsx
2018-10-29 08:48:13   30287320 FOIA - 504 (FY1991-Present).xlsx
2018-10-29 08:48:15   55120216 FOIA - 7(a)(FY1991-FY1999).xlsx
2018-10-29 08:48:17  116444478 FOIA - 7(a)(FY2000-FY2009).xlsx
2018-10-29 08:48:19   84757645 FOIA - 7(a)(FY2010-Present).xlsx
2018-10-26 03:13:21  179430516 SBAnational.csv
2018-11-02 04:14:18  262351663 SBAnational_new.csv
2018-11-06 09:32:50   26301182 bst_ex.dat
2018-11-06 09:32:49   26302165 clf_xgb.dat
2018-11-02 03:14:49  101980700 company_business_type.csv
2018-11-01 07:11:23   10955521 company_default_record.csv
2018-11-06 09:32:04    1896645 dict_categorical.pkl
2018-11-01 07:03:29   71184964 extra_company_info.csv
2018-10-29 03:22:11   10025457 loan_record.csv
2018-11-06 09:11:44    6861303 proj_bas.csv
2018-11-06 09:08:46    6405119 result_table_proj.csv
2018-10-29 02:42:40       1428 t0001-10.1080%2F10691898.2018.1434342.csv
2018-10-29 02:42:41      

In [8]:
import pickle

with fs.open(path_SBA + 'clf_xgb.dat', 'rb') as fp_in:
    clf_xgb = pickle.load(fp_in)
    
with fs.open(path_SBA + 'bst_ex.dat', 'rb') as fp_in:
    bst_ex = pickle.load(fp_in)
    
with fs.open(path_SBA + 'dict_categorical.pkl', 'rb') as fp_in:
    dict_categorical = pickle.load(fp_in);        

result_table_proj = pd.read_csv(path_SBA + 'result_table_proj.csv', sep = ';', low_memory = False)
proj_bas = pd.read_csv(path_SBA + 'proj_bas.csv', sep = ';', low_memory = False)


In [9]:
import eli5

In [10]:
eli5.show_weights(bst_ex, vec = dict_categorical, importance_type="weight")

Weight,Feature
0.1536,Zip5d_INT
0.1266,City_INT
0.0970,fips_INT
0.0768,Zip3d_INT
0.0730,Bank_INT
0.0663,NoEmp
0.0571,RetainedJob
0.0533,NAICS_group_INT
0.0472,NAICS_default_rate
0.0403,CreateJob


In [11]:
eli5.show_weights(bst_ex, vec = dict_categorical, importance_type="gain")

Weight,Feature
0.6440,RealEstate
0.0823,default_times
0.0499,RevLineCr_INT
0.0203,Bank_INT
0.0186,BankState_INT
0.0151,UrbanRural
0.0135,BusinessType_INT
0.0117,Previous_loan
0.0110,Loan_age
0.0092,Retaining_ratio_INT


In [12]:
eli5.show_weights(bst_ex, vec = dict_categorical, importance_type="cover")

Weight,Feature
0.3949,RealEstate
0.2309,default_times
0.0625,RevLineCr_INT
0.0545,Previous_loan
0.0338,Loan_age
0.0253,BusinessType_INT
0.0227,BankState_INT
0.0188,Bank_INT
0.0180,UrbanRural
0.0115,LowDoc_INT


In [13]:
result_table_proj.head()

,LoanNr_ChkDgt,Name,ApprovalFY,State,default,ChgOffPrinGr,GrAppv,SBA_Appv,SBA_ratio,prob,Grade
0,1075325003,"Gilly's, LLC",2005,FL,0,0.0,10000.0,5000.0,0.5,0.130445,3
1,1075335006,Gerald Christiansen dba Big G,2005,UT,0,0.0,36000.0,18000.0,0.5,0.090886,2
2,1075345009,Steven Wilson,2005,KY,0,0.0,10000.0,5000.0,0.5,0.068558,2
3,1075355001,Duraport Marine And Rail Termi,2005,NJ,0,0.0,100000.0,50000.0,0.5,0.056876,2
4,1075365004,Rucinsky's Painting and Decora,2005,WI,0,0.0,225000.0,112500.0,0.5,0.040396,1


In [15]:
result_table_proj[result_table_proj.default == 1].sample(5)

,LoanNr_ChkDgt,Name,ApprovalFY,State,default,ChgOffPrinGr,GrAppv,SBA_Appv,SBA_ratio,prob,Grade
32169,1470755003,"Building Relationships, LLC",2005,NH,1,10000.0,10000.0,5000.0,0.50,0.290047,5
41024,8166144009,SOUTHERN CROSS LAWN & MAINT.,2005,LA,1,4185.0,5000.0,4250.0,0.85,0.444351,5
30383,1447815010,Pakistan Television Network LL,2005,NY,1,24250.0,25000.0,12500.0,0.50,0.422587,5
10504,1200845006,"VALU MUFFLER OF KENMORE, INC.",2005,NY,1,92063.0,100000.0,50000.0,0.50,0.118343,3
10611,1202155004,"LEE GAS MART, INC.",2005,OH,1,45255.0,100000.0,50000.0,0.50,0.093486,2


In [16]:
eli5.show_prediction(bst_ex, proj_bas.iloc[32169], show_feature_values=True)